In [1]:
#coordinates left=-124.00055555649317, bottom=41.9994444436071, right=-122.99944444340576, top=43.00055555579519

In [9]:
import rasterio
from pyproj import Transformer
import numpy as np
import pandas as pd

# Path to your NLCD .img file
img_path = "nlcd_2021_land_cover_l48_20230630.img"

# Define your bounding box in WGS84 (lon_min, lon_max, lat_min, lat_max)
lon_min, lon_max = -124.00055555649317, -122.99944444340576   # Example longitude range
lat_min, lat_max = 41.9994444436071, 43.00055555579519      # Example latitude range

# Open the raster file
with rasterio.open(img_path) as dataset:
    # Get raster properties
    raster_crs = dataset.crs  # This is the Albers Conical Equal Area CRS
    raster_bounds = dataset.bounds
    print(f"Raster CRS: {raster_crs}")
    print(f"Raster Bounds: {raster_bounds}")

    # Transformer to convert from WGS84 (EPSG:4326) to the raster's CRS
    transformer = Transformer.from_crs("EPSG:4326", raster_crs, always_xy=True)

    # Convert bounding box (WGS84) to the raster’s coordinate system
    x_min, y_max = transformer.transform(lon_min, lat_max)  # Top-left corner
    x_max, y_min = transformer.transform(lon_max, lat_min)  # Bottom-right corner
    print(f"Converted Bounding Box in Raster CRS: ({x_min}, {y_min}) to ({x_max}, {y_max})")

    # Convert transformed coordinates to row and column indices
    row_max, col_min = dataset.index(x_min, y_min)  # Note: row_max here corresponds to y_min
    row_min, col_max = dataset.index(x_max, y_max)  # row_min corresponds to y_max
    print(f"Raw Indices: Row Min={row_min}, Row Max={row_max}, Col Min={col_min}, Col Max={col_max}")

    # Ensure row_min < row_max to avoid reversed slicing
    if row_min > row_max:
        row_min, row_max = row_max, row_min

    # Ensure indices are within raster bounds
    row_min = max(0, min(row_min, dataset.height - 1))
    row_max = max(0, min(row_max, dataset.height - 1))
    col_min = max(0, min(col_min, dataset.width - 1))
    col_max = max(0, min(col_max, dataset.width - 1))

    print(f"Adjusted Indices: Row Min={row_min}, Row Max={row_max}, Col Min={col_min}, Col Max={col_max}")

    # Read the raster data for the specified region
    land_cover_matrix = dataset.read(1)[row_min:row_max, col_min:col_max]

# # Convert the matrix into a Pandas DataFrame
# if land_cover_matrix.size > 0:
#     df = pd.DataFrame(land_cover_matrix)
#     csv_path = "land_cover_matrix.csv"
#     df.to_csv(csv_path, index=False)
#     print(f"Extracted land cover matrix saved to {csv_path}")
# else:
#     print("No data found in the specified region. Try adjusting the coordinates.")


Raster CRS: PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Raster Bounds: BoundingBox(left=-2493045.0, bottom=177285.0, right=2342655.0, top=3310005.0)
Converted Bounding Box in Raster CRS: (-2237834.863176505, 2424255.4883338944) to (-2191400.468043988, 2554847.887333584)
Raw Indices: Row Min=25171, Row Max=29524, Col Min=8507, Col Max=10054
Adjusted Indices: Row Min=25171, Row Max=29524, Col Min=8507, Col Max=10054


In [11]:
land_cover_matrix.shape

(4353, 1547)

In [12]:
land_cover_matrix

array([[43, 43, 21, ..., 81, 81, 81],
       [43, 43, 21, ..., 81, 81, 81],
       [43, 43, 21, ..., 81, 81, 81],
       ...,
       [71, 71, 52, ..., 42, 42, 42],
       [71, 52, 52, ..., 42, 42, 42],
       [52, 52, 52, ..., 42, 42, 42]], dtype=uint8)

In [18]:
# Get unique land cover classes
unique_classes = np.unique(land_cover_matrix)
num_classes = len(unique_classes)

print(f"Unique Land Cover Classes: {unique_classes}")
print(f"Original Matrix Shape: {land_cover_matrix.shape}")

# Initialize one-hot encoded matrix (num_classes, height, width)
one_hot_encoded = np.zeros((num_classes, *land_cover_matrix.shape), dtype=np.uint8)

# Populate the one-hot matrix
for i, category in enumerate(unique_classes):
    one_hot_encoded[i] = (land_cover_matrix == category).astype(np.uint8)

# Verify shape
print(f"One-hot Encoded Shape: {one_hot_encoded.shape}")  # Expected (10, 4353, 1547)


Unique Land Cover Classes: [11 21 22 23 24 31 41 42 43 52 71 81 82 90 95]
Original Matrix Shape: (4353, 1547)
One-hot Encoded Shape: (15, 4353, 1547)


In [21]:
# Save the one-hot encoded array as a NumPy file
np.save("NLCD2021_OR.npy", one_hot_encoded)

In [20]:
one_hot_encoded

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 